<html>
    <body style="background-color: red">
        <h1>Surviving the Titanic: A Machine Learning Approach to Predicting Passenger Survival</h1>
        <h2>Table of Contents</h2>
        <ol>
            <a href="#section1"><li>Importing libraries and loading the dataset</li></a>
            <a href="#section2"><li>Exploring the dataset</li></a>
            <a href="#section3"><li>Data cleaning</li></a>
            <a href="#section4"><li>Exploratory data analysis</li></a>
            <ol>
                <a href="#sub_section1_1"><li type="i">Univariate analysis</li></a>
                <a href="#sub_section1_2"><li type="i">Bivariate analysis</li></a>
            </ol>        
            <a href="#section5"><li>Data Prepocessing</li></a>
            <a href="#section6"><li>Model Building and Evaluation</li></a>
            <ol>
                <a href="#sub_section2_1"><li type="i">KNN Classifier</li></a>
                <a href="#sub_section2_2"><li type="i">Logistic Regreassion</li></a>
                <a href="#sub_section2_3"><li type="i">Decision Tree Classifier</li></a>
                <a href="#sub_section2_4"><li type="i">MPLC Classifier</li></a>
            </ol> 
    </body>
</html>

<h3 id="section1">1. Importing libraries and loading the dataset</h3>
<p>Let's start by importing the necessary libraries and loading the dataset.</p>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tabulate import tabulate
import warnings

color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

warnings.filterwarnings(action = 'ignore')


In [ ]:
# Read the data
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_submission  = pd.read_csv('gender_submission.csv')

<h3 id="section2">2. Exploring the dataset</h3>
<p>Let's explore the datasets:</p>

In [ ]:
# Shape of the data
print('Shape of the train data: %s', df_train.shape)
print('Shape of the test data: %s', df_test.shape)
print('Shape of the submission data: %s', df_submission.shape)

In [ ]:
# Sample train data
df_train.head()

In [ ]:
# Smaple of test data
df_test.head()

In [ ]:
# Sample submission data
df_submission.head()

<p>Let's explore the train dataset to get a better understanding of its structure and content:</p>

In [ ]:
# Data types
df_train.dtypes

In [ ]:
# Summary statistics
df_train.describe()

In [ ]:
# Unique values in each column
df_train.nunique()

<h3 id="section3">3. Data cleaning</h3>
<p>Nice! We have a dataset with <b>891</b> rows and <b>12</b> columns. Let's clean the dataset by handling missing values, duplicates, irrelevant columns, and converting data types.</p>

In [ ]:
# Let's create a copy of the train and test data to perform data cleaning
df_train_copy = df_train.copy()
df_test_copy = df_test.copy()

In [ ]:
# Missing values in train data
df_train_copy.isna().sum()

In [ ]:
# Missing values in test data
df_test_copy.isna().sum()

<p>We have ver high number of missing values in <b>Cabin</b> followed by <b>Age</b> and just 2 in <b>Embarked</b> column.</p>
<p>Lets impute missing values in <b>Age</b> column by median and <b>Embarked</b> column by mode.</p>

In [ ]:
# Impute missing values in Age column with median
df_train_copy['Age'] = df_train_copy['Age'].fillna(df_train_copy['Age'].median())
df_test_copy['Age'] = df_test_copy['Age'].fillna(df_test_copy['Age'].median())

# Impute missing values in Embarked column with mode
df_train_copy['Embarked'] = df_train_copy['Embarked'].fillna(df_train_copy['Embarked'].mode()[0])
df_test_copy['Embarked'] = df_test_copy['Embarked'].fillna(df_test_copy['Embarked'].mode()[0])


<p>Here empty values in <b>Cabin</b> column may indicate that passenger didn't have a cabin. So we can explore further if emplty values in cabin has relation with Survived</p>

In [ ]:
# Check if there is any relations between the missing values in Cabin column and Survived column
df_train_copy[df_train_copy['Cabin'].isna()]['Survived'].value_counts()

<p>We can clearly see that passengers with empty values in <b>Cabin</b> column have less chance of survival. So we can impute empty values in <b>Cabin</b> column with <b>Missing</b>.</p>

In [ ]:
# Impute missing values in Cabin column with 'Missing'
df_train_copy['Cabin'] = df_train_copy['Cabin'].fillna('Missing')
df_test_copy['Cabin'] = df_test_copy['Cabin'].fillna('Missing')

In [ ]:
# Let see if we still have any missing values in the train data
df_train_copy.isna().sum()

In [ ]:
# Imputing missing values in Fare column with median in test data
df_test_copy['Fare'] = df_test_copy['Fare'].fillna(df_test_copy['Fare'].median())

# Let see if we still have any missing values in test data
df_test_copy.isna().sum()

In [ ]:
# Check for duplicates in train data
df_train_copy.duplicated().sum()

In [ ]:
# Check for duplicates in test data
df_test_copy.duplicated().sum()

<p>No duplicates in the dataset! Let's move on to the next step.</p>

<p>Let's convert Survived, Pclass, Sex, SibSp, Parch, and Embarked to categorical variables.</p>

In [ ]:
# Convert Name, Survived, Pclass, Sex, SibSp, Parch, Embarked, Ticket, Cabin to categorical variables in train data
df_train_copy['Survived'] = df_train_copy['Survived'].astype('category')
df_train_copy['Pclass'] = df_train_copy['Pclass'].astype('category')
df_train_copy['Sex'] = df_train_copy['Sex'].astype('category')
df_train_copy['SibSp'] = df_train_copy['SibSp'].astype('category')
df_train_copy['Parch'] = df_train_copy['Parch'].astype('category')
df_train_copy['Embarked'] = df_train_copy['Embarked'].astype('category')
df_train_copy['Ticket'] = df_train_copy['Ticket'].astype('category')
df_train_copy['Cabin'] = df_train_copy['Cabin'].astype('category')
df_train_copy['Name'] = df_train_copy['Name'].astype('category')


In [ ]:
# Convert Name, Pclass, Sex, SibSp, Parch, Embarked, Ticket, Cabin to categorical variables in test data
df_test_copy['Pclass'] = df_test_copy['Pclass'].astype('category')
df_test_copy['Sex'] = df_test_copy['Sex'].astype('category')
df_test_copy['SibSp'] = df_test_copy['SibSp'].astype('category')
df_test_copy['Parch'] = df_test_copy['Parch'].astype('category')
df_test_copy['Embarked'] = df_test_copy['Embarked'].astype('category')
df_test_copy['Ticket'] = df_test_copy['Ticket'].astype('category')
df_test_copy['Cabin'] = df_test_copy['Cabin'].astype('category')
df_test_copy['Name'] = df_test_copy['Name'].astype('category')

<h3 id="section4">4. Exploratory data analysis</h3>
<p>Let's perform exploratory data analysis to extract insights from the cab trips dataset:</p>
<h4 id="sub_section1" >i. Univariate analysis</h4>
<p>We will start by exploring the distribution of the numerical and categorical variables in the dataset:</p>

In [ ]:
# Function for plolting the distribution of categorical variables
def plot_cat(df, col, x_label, y_label, plot_title):
    sns.set_style('darkgrid')
    plt.figure(figsize=(12,8))
    sns.countplot(data=df, x=col, color='dodgerblue')
    plt.title(plot_title, fontsize=14)
    plt.xlabel(x_label, fontsize=12)
    plt.ylabel(y_label, fontsize=12)
    plt.show()

<p>Here Survived column is our target variable. Let's explore its distribution.</p>

In [ ]:
# Plot distribution of Survived column
plot_cat(df_train_copy, 'Survived', 'Survived', 'Count', 'Distribution of Survived column')

In [ ]:
# Plotting distribution of Pclass column
plot_cat(df_train_copy, 'Pclass', 'Pclass', 'Count', 'Distribution of Pclass column')

In [ ]:
# Plotting distribution of Sex column
plot_cat(df_train_copy, 'Sex', 'Sex', 'Count', 'Distribution of Sex column')

In [ ]:
# Plottting distribution of SibSp column
plot_cat(df_train_copy, 'SibSp', 'SibSp', 'Count', 'Distribution of SibSp column')

In [ ]:
# Plottting distribution of Parch column
plot_cat(df_train_copy, 'Parch', 'Parch', 'Count', 'Distribution of Parch column')

In [ ]:
# Plottting distribution of Embarked column
plot_cat(df_train_copy, 'Embarked', 'Embarked', 'Count', 'Distribution of Embarked column')

<p>Lets explore the distribution of numerical variables.</p>

In [ ]:
# Function for calculating descriptives of numeric variable and plotting the distribution
def plot_dist(df, col, x_label, y_label, plot_title):
    _min = df[col].min()
    _max = df[col].max()
    ran = df[col].max()-df[col].min()
    mean = df[col].mean()
    median = df[col].median()
    st_dev = df[col].std()
    skew = df[col].skew()
    kurt = df[col].kurtosis()

    # calculating points of standard deviation
    points = mean-st_dev, mean+st_dev
    sns.set_style('darkgrid')
    plt.figure(figsize=(12,8))
    sns.histplot(data=df, x=col, bins=30, kde=True, color='dodgerblue')
    sns.lineplot(x=points, y=[0,0], color = 'black', label = "std_dev")
    sns.scatterplot(x=[_min,_max], y=[0,0], color = 'orange', label = "min/max")
    sns.scatterplot(x=[mean], y=[0], color = 'red', label = "mean")
    sns.scatterplot(x=[median], y=[0], color = 'blue', label = "median")
    plt.title(plot_title, fontsize=14)
    plt.xlabel(x_label, fontsize=12)
    plt.ylabel(y_label, fontsize=12)

    # Creating a DataFrame for the descriptive statistics
    variable_stats = pd.DataFrame({'Statistics': ['Minimum Value', 'Maximum Value', 'Range', 'Mean', 
                                                    'Median', 'Standard Deviation', 'Skewness', 'Kurtosis'], 
                                        'Value': [_min, _max, ran, mean, median, st_dev, skew, kurt]})
    
    plt.show()

    display(tabulate(variable_stats, headers='keys', showindex=False, tablefmt='html'))


In [ ]:
# Plotting distribution of Age column
plot_dist(df_train, 'Age', 'Age', 'Count', 'Distribution of Age column')

<ul>
    <li>Median Age is 28.</li>
    <li>Mean and Median Age are almost same. So Age is normally distributed.</li>
</ul>

In [ ]:
# Let's see how Fare column is distributed
plot_dist(df_train, 'Fare', 'Fare', 'Count', 'Distribution of Fare column')

<p>Kurtosis of Fare is very high. So Fare is highly skewed.</p>
<p>Which is as expected, because certain classes will have higher fares than others and limited seats.</p>

<h4 id="sub_section2">ii. Bivariate analysis</h4>
<p>Let's explore the relationship between the trip duration and other variables in the dataset:</p>

In [ ]:
# Function for plotting the distribution of numeric variables against the target variable
# Here target variable is assumed to be categorical
def plot_num_vs_target(df, col, target, x_label, y_label, plot_title):
    sns.set_style('darkgrid')
    plt.figure(figsize=(12,8))
    sns.boxplot(data=df, x=target, y=col, color='dodgerblue')
    plt.title(plot_title, fontsize=14)
    plt.xlabel(x_label, fontsize=12)
    plt.ylabel(y_label, fontsize=12)
    plt.show()

In [ ]:
# Relationship between Survived and Age
plot_num_vs_target(df_train_copy, 'Age', 'Survived', 'Survived', 'Age', 'Relationship between Survived and Age')

In [ ]:
# Relationship between Survived and Fare
plot_num_vs_target(df_train_copy, 'Fare', 'Survived', 'Survived', 'Fare', 'Relationship between Survived and Fare')

In [ ]:
# Function for plotting the distribution of categorical variables against the target variable
# Here target variable and categorical variable are assumed to be categorical
def plot_cat_vs_target(df, col, target, x_label, y_label, plot_title):
    sns.set_style('darkgrid')
    plt.figure(figsize=(12,8))
    sns.countplot(data=df, x=col, hue=target, palette='Set1')
    plt.title(plot_title, fontsize=14)
    plt.xlabel(x_label, fontsize=12)
    plt.ylabel(y_label, fontsize=12)
    plt.show()

In [ ]:
# Relationship between Survived and Pclass
plot_cat_vs_target(df_train_copy, 'Pclass', 'Survived', 'Pclass', 'Count', 'Relationship between Survived and Pclass')

In [ ]:
# Relationship between Survived and Sex
plot_cat_vs_target(df_train_copy, 'Sex', 'Survived', 'Sex', 'Count', 'Relationship between Survived and Sex')

In [ ]:
# Relationship between Survived and SibSp
plot_cat_vs_target(df_train_copy, 'SibSp', 'Survived', 'SibSp', 'Count', 'Relationship between Survived and SibSp')

In [ ]:
# Relationship between Survived and Parch
plot_cat_vs_target(df_train_copy, 'Parch', 'Survived', 'Parch', 'Count', 'Relationship between Survived and Parch')

In [ ]:
# Relationship between Survived and Embarked
plot_cat_vs_target(df_train_copy, 'Embarked', 'Survived', 'Embarked', 'Count', 'Relationship between Survived and Embarked')

<h3 id="section5">5. Data Preprocessing</h3>
<p>Before we use variables in our model, we need to preprocess them. We will perform the following steps:</p>
<ul>
    <li>One-hot encode categorical variables</li>
    <li>Lable encode categorical variables</li>
</ul>

In [ ]:
# Function to encode categorical variables, we will use scikit-learn's LabelEncoder for label encoding and pandas get_dummies for one-hot encoding
from sklearn.preprocessing import LabelEncoder

def encode_cat(df, col, encoding_type):
    if encoding_type == 'label':
        label_encoder = LabelEncoder()
        df[col] = label_encoder.fit_transform(df[col])
    elif encoding_type == 'onehot':
        df = pd.get_dummies(df, columns=[col], prefix=[col])
    return df

In [ ]:
# Encoding variables in the training dataset and create a new dataframe called df_train_encoded
df_train_encoded = df_train_copy.copy()
df_train_encoded = encode_cat(df_train_encoded, 'Survived', 'label')
df_train_encoded = encode_cat(df_train_encoded, 'Cabin', 'label')
df_train_encoded = encode_cat(df_train_encoded, 'Pclass', 'label')
df_train_encoded = encode_cat(df_train_encoded, 'Sex', 'label')
df_train_encoded = encode_cat(df_train_encoded, 'SibSp', 'label')
df_train_encoded = encode_cat(df_train_encoded, 'Parch', 'label')
df_train_encoded = encode_cat(df_train_encoded, 'Embarked', 'label')

# Encoding variables in the test dataset and create a new dataframe called df_test_encoded
df_test_encoded = df_test_copy.copy()
df_test_encoded = encode_cat(df_test_encoded, 'Cabin', 'label')
df_test_encoded = encode_cat(df_test_encoded, 'Pclass', 'label')
df_test_encoded = encode_cat(df_test_encoded, 'Sex', 'label')
df_test_encoded = encode_cat(df_test_encoded, 'SibSp', 'label')
df_test_encoded = encode_cat(df_test_encoded, 'Parch', 'label')
df_test_encoded = encode_cat(df_test_encoded, 'Embarked', 'label')

In [ ]:
# Check the processed training set
df_train_encoded.head()

In [ ]:
# Check the processed test set
df_test_encoded.head()

<p>Now that we have preprocessed the variables, let's check the correlation between them:</p>

In [ ]:
# Function to plot correlation between variables
def plot_corr(df, size=10):
    corr = df.corr()
#     print(corr)
    fig, ax = plt.subplots(figsize=(size, size))
    sns.heatmap(corr, annot=True, linewidths=.5, ax=ax, cmap='crest')
    plt.show() 

In [ ]:
# Correlation between variables in the training set
plot_corr(df_train_encoded.drop(['PassengerId', 'Name', 'Ticket'], axis=1))

<p>Let's check the correlation between the variables and the target variable:</p>

In [ ]:
# Function to plot correlation of variables with the target variable as a barplot
def plot_corr_target(df, target, size=10):
    corr = df.corr()
    corr_target = corr[target]
    corr_target = corr_target.sort_values(ascending=False)
    corr_target = corr_target.drop(target)
    plt.figure(figsize=(size, size))
    corr_target.plot.barh()
    plt.show()

In [ ]:
# Check correlation of variables with the target variable
plot_corr_target(df_train_encoded.drop(['Name', 'Ticket', 'PassengerId'], axis=1), 'Survived')

<h3 id="section6">6. Model Building and Evaluation</h3>
<p>Let's build a model to predict the Survival of passengers on the Titanic:</p>
<p>We will use different classification algorithms to build models and evaluate them using F1 score:</p>

In [ ]:
# We will first separate the target variable from the features
y = df_train_encoded['Survived']
x = df_train_encoded.drop(['Survived', 'Name', 'Ticket', 'PassengerId'], axis=1)
x.shape, y.shape

<p>Let's scale the features usinf scikit-learn's MinMax scaler:</p>

In [ ]:
## Importing the MinMax Scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)

In [ ]:
x = pd.DataFrame(x_scaled, columns = x.columns)

In [ ]:
# Check data after scaling
x.head()

<p>Now, let's split the dataset into training and test sets:</p>

In [ ]:
# Importing the train test split function
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(x,y, random_state = 56, stratify=y)

<h4 id="sub_section2_1">i. KNN Classifier</h4>

In [ ]:
# Import KNN classifier and metric F1 score
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import f1_score

In [ ]:
# Creating instance of KNN
clf = KNN(n_neighbors = 10)

# Fitting the model
clf.fit(train_x, train_y)

# Predicting over the Train Set and calculating F1
test_predict = clf.predict(test_x)
k = f1_score(test_predict, test_y)
print('Test F1 Score    ', k )

<p>Now, let's use Elbow method to find the optimal value of K:</p>

In [ ]:
# Function to find the best value of K
def Elbow(K):
    #initiating empty list
    test_error = []
   
    #training model for evey value of K
    for i in K:
        #Instance oh KNN
        clf = KNN(n_neighbors = i)
        clf.fit(train_x, train_y)
        # Appending F1 scores to empty list claculated using the predictions
        tmp = clf.predict(test_x)
        tmp = f1_score(tmp,test_y)
        error = 1-tmp
        test_error.append(error)
    
    return test_error

In [ ]:
#Defining K range
k = range(6, 20, 2)

In [ ]:
# calling above defined function
test = Elbow(k)

In [ ]:
# plotting the Curves
plt.plot(k, test)
plt.xlabel('K Neighbors')
plt.ylabel('Test error')
plt.title('Elbow Curve for test')

In [ ]:
# Creating instance of KNN
clf = KNN(n_neighbors = 6)

# Fitting the model
clf.fit(train_x, train_y)

# Predicting over the Test Set and calculating F1
test_predict = clf.predict(test_x)
k = f1_score(test_predict, test_y)
print('Test F1 Score    ', k )

In [ ]:
submission_predictions = clf.predict(df_test_encoded.drop(['Name', 'Ticket', 'PassengerId'], axis=1))

In [ ]:
submission_predictions

In [ ]:
df_submission['Survived'] = submission_predictions

In [ ]:
df_submission.head()

In [ ]:
df_submission.to_csv('submission.csv', index=False)

<h4 id="sub_section2_2">ii. Logistic Regression</h4>

In [ ]:
# Importing Logistic Regression
from sklearn.linear_model import LogisticRegression

In [ ]:
# Creating instance of Logistic Regression
log_reg = LogisticRegression()

# Fitting the model
log_reg.fit(train_x, train_y)

# Predicting over the Test Set and calculating F1
test_predict_log = log_reg.predict(test_x)
k_log = f1_score(test_predict_log, test_y)

print('Test F1 Score    ', k_log )

In [ ]:
submission_predictions_log = log_reg.predict(df_test_encoded.drop(['Name', 'Ticket', 'PassengerId'], axis=1))

In [ ]:
submission_predictions_log

In [ ]:
# Combine predics with df_submission and save to csv
df_submission['Survived'] = submission_predictions_log
df_submission.to_csv('submission_log.csv', index=False)

<h4 id="sub_section2_3">iii. Decision Tree Classifier</h4>

In [ ]:
# Importing Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Creating instance of Decision Tree Classifier
clf = DecisionTreeClassifier()

# Fitting the model
clf.fit(train_x, train_y)

# Predicting over the Test Set and calculating F1
test_predict_dt = clf.predict(test_x)
k_dt = f1_score(test_predict_dt, test_y)

print('Test F1 Score    ', k_dt )

In [ ]:
submission_predictions_dt = clf.predict(df_test_encoded.drop(['Name', 'Ticket', 'PassengerId'], axis=1))

# 

In [ ]:
# Combine predictions with df_submission and save to csv
df_submission['Survived'] = submission_predictions_dt
df_submission.to_csv('submission_dt.csv', index=False)

<h4 id="sub_section2_4">iv. MLPClassifier</h4>

In [ ]:
# Importing MLPC Classifier
from sklearn.neural_network import MLPClassifier

In [ ]:
# Creating instance of MLPC Classifier
clf = MLPClassifier()

# Fitting the model
clf.fit(train_x, train_y)

# Predicting over the Test Set and calculating F1
test_predict_mlpc = clf.predict(test_x)
k_mlpc = f1_score(test_predict_mlpc, test_y)

print('Test F1 Score    ', k_mlpc )

In [ ]:
submission_predictions_mlpc = clf.predict(df_test_encoded.drop(['Name', 'Ticket', 'PassengerId'], axis=1))

In [ ]:
# Combine predictions with df_submission and save to csv
df_submission['Survived'] = submission_predictions_mlpc
df_submission.to_csv('submission_mlpc.csv', index=False)